In [1]:
import hashlib
from datetime import datetime


class Block:
    """
        区块结构
            prev_hash:      父区块哈希值
            data:           区块内容
            timestamp:      区块创建时间
            hash:           区块哈希值
            Nonce:        随机数
    """
    def __init__(self, data, prev_hash):
        # 将传入的父哈希值和数据保存到类变量中
        self.prev_hash = prev_hash    
        self.data = data
        # 获取当前时间
        self.timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # 设置Nonce和哈希的初始值为None
        self.nonce = None
        self.hash = None
        
    def __repr__(self):
        return "区块内容：%s\n哈希值: %s" % (self.data, self.hash)

In [2]:
class ProofOfWork:
    """
        工作量证明
    """
    def __init__(self, block, difficult=5):
        self.block = block
        
        # 定义工作量难度，默认为5，表示有效的哈希值以5个“0”开头
        self.difficulty = difficult

    def mine(self):
        """
            挖矿函数
        """
        i = 0
        prefix = '0' * self.difficulty

        while True:
            message = hashlib.sha256()
            message.update(str(self.block.prev_hash).encode('utf-8'))
            message.update(str(self.block.data).encode('utf-8'))
            message.update(str(self.block.timestamp).encode('utf-8'))
            message.update(str(i).encode("utf-8"))
            digest = message.hexdigest()
            if digest.startswith(prefix):
                self.block.nonce = i
                self.block.hash = digest
                return self.block
            i += 1

    def validate(self):
        """
            验证有效性
        """
        message = hashlib.sha256()
        message.update(str(self.block.prev_hash).encode('utf-8'))
        message.update(str(self.block.data).encode('utf-8'))
        message.update(str(self.block.timestamp).encode('utf-8'))
        message.update(str(self.block.nonce).encode('utf-8'))
        digest = message.hexdigest()

        prefix = '0' * self.difficulty
        return digest.startswith(prefix)

In [3]:
# 先定义一个区块

b = Block(data="测试", prev_hash="")

# 再定义一个工作量证明
w = ProofOfWork(b)

In [4]:
# 进行挖矿，并统计函数执行时间
%time  valid_block= w.mine() 

CPU times: user 1.42 s, sys: 2.95 ms, total: 1.42 s
Wall time: 1.42 s


In [5]:
# 验证区块，并计算执行时间
%time  w.validate()

CPU times: user 20 µs, sys: 0 ns, total: 20 µs
Wall time: 26 µs


True

In [7]:
class BlockChain:
    """
        区块链结构体
            blocks:        包含的区块列表
    """
    def __init__(self):
        self.blocks = []

    def add_block(self, block):
        """
        添加区块
        """
        self.blocks.append(block)

In [8]:
blockchain = BlockChain()

new_block1 = Block(data="创世区块", prev_hash="")
w1 = ProofOfWork(new_block1)
genesis_block = w1.mine()
blockchain.add_block(genesis_block)

new_block2 = Block(data="张三转给李四1个比特币", prev_hash=genesis_block.hash)
w2 = ProofOfWork(new_block2)
new_block = w2.mine()
blockchain.add_block(new_block)

new_block3 = Block(data="张三转给王五2个比特币", prev_hash=new_block.hash)
w3 = ProofOfWork(new_block3)
new_block = w3.mine()
blockchain.add_block(new_block)

In [10]:
# 打印区块链

print("区块链包含区块个数: %d\n" % len(blockchain.blocks))

for block in blockchain.blocks:
    print("上一个区块哈希：%s" % block.prev_hash)
    print("区块内容：%s" % block.data)
    print("区块哈希：%s" % block.hash)
    print("\n") 

区块链包含区块个数: 3

上一个区块哈希：
区块内容：创世区块
区块哈希：00000d8129023b1740e154e63e820a07b6f603d71f231257fe73e2cfffd2f036


上一个区块哈希：00000d8129023b1740e154e63e820a07b6f603d71f231257fe73e2cfffd2f036
区块内容：张三转给李四1个比特币
区块哈希：00000d38ce6fa764dcd1951e8e6a4ecaa333f0576180f9df4fcd407d9409899c


上一个区块哈希：00000d38ce6fa764dcd1951e8e6a4ecaa333f0576180f9df4fcd407d9409899c
区块内容：张三转给王五2个比特币
区块哈希：00000d302a8304000ce5d0bb3f14af90a5178182b68861c1bab24c9e4f2e189e


